In [6]:
import os, shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
src_dir = "/content/dataset"  # where 0/ and 1/ are currently
base_dir = "/content/split_dataset"

# Make directories
for split in ["train", "val", "test"]:
    for label in ["0", "1"]:
        os.makedirs(os.path.join(base_dir, split, label), exist_ok=True)

# Function to split and copy images
def split_and_copy(class_name):
    imgs = os.listdir(os.path.join(src_dir, class_name))
    imgs = [f for f in imgs if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    train_imgs, temp_imgs = train_test_split(imgs, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "train", class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "val", class_name, img))
    for img in test_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "test", class_name, img))

# Split both classes
split_and_copy("0")
split_and_copy("1")



  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 2.1/332.0 MB 10.9 MB/s eta 0:00:31
    --------------------------------------- 4.2/332.0 MB 11.0 MB/s eta 0:00:30
    --------------------------------------- 6.6/332.0 MB 10.9 MB/s eta 0:00:30
   - -------------------------------------- 9.2/332.0 MB 11.0 MB/s eta 0:00:30
   - -------------------------------------- 11.5/332.0 MB 11.0 MB/s eta 0:00:30
   - -------------------------------------- 13.6/332.0 MB 11.0 MB/s eta 0:00:30
   - -------------------------------------- 16.3/332.0 MB 11.0 MB/s eta 0:00:29
   -- ------------------------------------- 18.6/332.0 MB 11.0 MB/s eta 0:

In [ ]:
# ==========================
# 1. IMPORT LIBRARIES
# ==========================
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# ==========================
# 2. DATA LOADING & AUGMENTATION
# ==========================
# Set paths (change these to your local or Kaggle dataset directories)
train_dir = r'C:\Users\tarun\dataset\training'
val_dir = r'C:\Users\tarun\dataset\validation'
test_dir = r'C:\users\tarun\dataset\training'

# Data augmentation for training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Validation & test sets: only rescale
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=(128,128),
                                              batch_size=32,
                                              class_mode='binary')

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=(128,128),
                                          batch_size=32,
                                          class_mode='binary')

test_gen = val_datagen.flow_from_directory(test_dir,
                                           target_size=(128,128),
                                           batch_size=32,
                                           class_mode='binary',
                                           shuffle=False)

# ==========================
# 3. BUILD CNN MODEL
# ==========================
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ==========================
# 4. TRAIN MODEL
# ==========================
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=50,
                    callbacks=[early_stop],
                    verbose=1)

# ==========================
# 5. EVALUATE MODEL
# ==========================
loss, acc = model.evaluate(test_gen, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")

# ==========================
# 6. PLOT TRAINING HISTORY
# ==========================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')

plt.tight_layout()
plt.show()


Found 423742 images belonging to 221 classes.
Found 54256 images belonging to 39 classes.
Found 423742 images belonging to 221 classes.


C:\ana\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ana\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
   26/13242 ━━━━━━━━━━━━━━━━━━━━ 1:29:51 408ms/step - accuracy: 0.1523 - loss: -195477.9844

In [9]:
X


array([[[ 1.09706398],
        [-2.07333501],
        [ 1.26993369],
        ...,
        [ 2.29607613],
        [ 2.75062224],
        [ 1.93701461]],

       [[ 1.82982061],
        [-0.35363241],
        [ 1.68595471],
        ...,
        [ 1.0870843 ],
        [-0.24388967],
        [ 0.28118999]],

       [[ 1.57988811],
        [ 0.45618695],
        [ 1.56650313],
        ...,
        [ 1.95500035],
        [ 1.152255  ],
        [ 0.20139121]],

       ...,

       [[ 0.70228425],
        [ 2.0455738 ],
        [ 0.67267578],
        ...,
        [ 0.41406869],
        [-1.10454895],
        [-0.31840916]],

       [[ 1.83834103],
        [ 2.33645719],
        [ 1.98252415],
        ...,
        [ 2.28998549],
        [ 1.91908301],
        [ 2.21963528]],

       [[-1.80840125],
        [ 1.22179204],
        [-1.81438851],
        ...,
        [-1.74506282],
        [-0.04813821],
        [-0.75120669]]])